# DNBLab Jupyter Notebook Tutorial

## SRU-Abfragen erklärt - Tutorial für Einsteiger, die es wissen wollen

Dieses Tutorial erklärt die Möglichkeiten, mit Hilfe von Jupyter Notebooks und Python die SRU-Schnittstelle der DNB abzufragen und mit den erhaltenen Antworten zu arbeiten. Das Tutorial erklärt dazu den Aufbau der Abfragen anhand von Beispielen und stützt sich auf die Dokumentation der SRU-Schnittstelle unter https://www.dnb.de/sru.

Das Tutorial ist dazu in folgendermaßen aufgebaut: 

* [1. Einrichten der Arbeitsumgebung](#Teil1) 
* [2. Abfragen verschiedener Datensätze der DNB](#Teil2)  
* [3. Aufbau einer gezielten Suche](#Teil3) 
* [4. Verarbeiten der Ergebnisse](#Teil4)


### 1. Einrichten der Arbeitsumgebung  <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, importiren wird zunächst die benötigten Python-Biblitoheken: Wir laden dazu "requests" für die Abfragen an die SRU-Schnittstelle, ElementTree (als ET) und BeautifulSoup, um unser MARC21-XML analysieren zu können, unicodedata zur Codierung sowie Pandas (als pd) zur Datenmanipulation und -analyse.: 

In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

#URL der SRU-Schnittstelle der DNB: 
base_url = "https://services.dnb.de/sru"

#Anfrage - wir speichern das Ergebnis in die Variable "basic_request":
basic_request = requests.get(base_url)

Die Antwort der Schnittstelle können wir uns auf verschiedenen Wegen ansehen: So können wir uns bspw. den übergebenen Inhalt als Text ausgeben lassen. Dies ist vor allem dann sinnvoll, wenn wir noch nicht wissen, welches Format die Schnittstelle per default ausliefert. Wenn wir dies bereits wissen, zuvor abgefragt haben oder bereits in unserer Abfrage spezifiert haben (dazu später mehr), können wir uns den Inhalt auch direkt in das passende Format konvertieren lassen. In diesem Fall gibt die SRU-Schnittstelle zunächst HTML zurück:  

In [2]:
#Ausgabe der ursprünglichen Abfrage als Text: 
print(basic_request.text)

<html>
<body>
<h1>SRU is running.</h1>
Version: 2.29, Build: 2021-02-24T12:42:52 UTC
</body>
</html>



### 2. Abfragen verschiedener Datensätze der DNB  <a class="anchor" id="Teil2"></a>

Wir können nun unterschiedliche Bereiche der DNB-Daten abfragen, indem wir die oben definierte Base-URL erweitern. Mölgich sind über die SRU-Schnittstelle Abfragen drei verschiedener Bereiche: 

* Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
* Katalog des Deutschen Musikarchivs (DMA)
* Katalog der Gemeinsamen Normdatei (GND)

Die erweiterungen für die URL sind dabei folgende: 

* DNB: https://services.dnb.de/sru/dnb
* DMA: https://services.dnb.de/sru/dnb.dma
* GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine "Explain-Response" in XML zurück. Mit Hilfe der Programmbibliothek BeautifulSoup "parsen" wir die Antwort in XML, d.h. wir wandeln diese Antwort in XML um, und lassen uns diese anzeigen. Achtung: Um die Antwort nicht zu lang werden zu lassen, werden hier nur die ersten 500 Zeichen der Antwort ausgegeben - die eigentliche Antwort ist länger und kann durch einfaches Löschen der Einschränkung "[0:500]" in der "print"-Zeile komplett angezeigt werden. Natürlich können auch andere Bereiche zur Anzeige gewählt werden. 

In [3]:
dnb = requests.get("https://services.dnb.de/sru/dnb")

response = soup(dnb.content)
print(response.prettify()[0:500])


<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <explainresponse xmlns="http://www.loc.gov/zing/srw/">
   <version>
    1.1
   </version>
   <record>
    <recordschema>
     http://explain.z3950.org/dtd/2.0/
    </recordschema>
    <recordpacking>
     xml
    </recordpacking>
    <recorddata>
     <ns:explain id="Deutsche Nationalbibliothek" xmlns:ns="http://explain.z3950.org/dtd/2.0/">
      <ns:serverinfo protocol="sru" version="1.1">
       <ns:host>
        services.dnb.de
       </


Wenn wir nun eine Suchanfrage im DNB-Katalog vornehmen wollen, definieren wir zunächst über die Wahl der URL unseren Suchbereich. Mit Hilfe der Variable "parameter" übergeben wir dann all die Werte, die wir für unsere Suchanfrage per SRU brauchen. 

Besonders relevant sind im Folgenden dabei die beiden Punkte 'query' : 'Klimawandel', sowie 'recordSchema' : 'MARC21-xml'. Statt "Klimawandel" kann hier jeder beliebige Suchbegriff eingetragen werden - auch Suchbegriffe, die aus mehreren Wörtern bestehen, können hier mittels boolscher Operatoren übergeben werden. Wie genau solche Anfragen formuliert werden müssen, kann unter https://www.dnb.de/sru nachgelesen werden.

In [4]:
dnb_url = "https://services.dnb.de/sru/dnb"

#Parameter, die wir mit einer Anfrage übergeben wollen: 
parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : 'Klimawandel', 'recordSchema' : 'MARC21-xml'} 

r1 = requests.get(dnb_url, params = parameter)

#print(r.url)

response = soup(r1.content)
print(response.prettify()[0:1000])


<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <searchretrieveresponse xmlns="http://www.loc.gov/zing/srw/">
   <version>
    1.1
   </version>
   <numberofrecords>
    6018
   </numberofrecords>
   <records>
    <record>
     <recordschema>
      MARC21-xml
     </recordschema>
     <recordpacking>
      xml
     </recordpacking>
     <recorddata>
      <record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
       <leader>
        00000nam a22000008c 4500
       </leader>
       <controlfield tag="001">
        1147699615
       </controlfield>
       <controlfield tag="003">
        DE-101
       </controlfield>
       <controlfield tag="005">
        20201001130612.0
       </controlfield>
       <controlfield tag="007">
        tu
       </controlfield>
       <controlfield tag="008">
        171204s2025    gw ||||| |||| 00||||ger
       </controlfield>
       <datafield ind1=" " ind2=" " tag="015">
        <subfield code="a">
         17,N50
        </subfie

Zu beachten ist, dass die Suche nach einem Stichwort über den 'query'-Befehl eine allgemeine Suche über alle Titeldaten darstellt. Die Suche ist allerdings nicht auf Titel oder ähnliches beschränkt, sondern durchsucht die Datensätze im Gesamten. Auch nach beispielsweise Autor\*innennamen kann auf diese Art gesucht werden, jedoch muss bedacht werden, dass auch Titel, die den gesuchten Namen enthalten, in diesem Fall als Treffer ausgegeben werden. 

### 3. Aufbau einer gezielten Suche  <a class="anchor" id="Teil3"></a>

Um die Suche direkt auf bestimmte Angaben wie bspw. Titel oder Autor\*in einzugrenzen, können unter anderem folgende Befehle genutzt werden:

 - tit= Suche im Titeleintrag
 - atr= Suche nach Verfasser\*in (Person oder Organisation)
 - per= Suche nach Personen (in allen relevanten Feldern)
 - sw = Suche nach Schlagworten
 - jhr = Suche nach Erscheinungszeitraum 
 - ...

Eine detaillierte Übersicht über die verschiedenen Abfragemöglichkeiten gibt es hier: https://www.dnb.de/expertensuche. Dabei können die unterschiedlichen Parameter auch beliebig in der Suchanfrage kombiniert werden - zu beachten ist hier lediglich, dass diese immer Teil der "Query" sind. 

Für die Ausgabe der Ergebnisse können wir außerdem zwischen drei Formaten wählen, indem wir den entsprechenden Code hinter 'recordSchema' ändern:  

 - MARC21-xml (XML-Variante von MARC 21)
 - oai_dc (DNB Casual - Auswahl von Dublin-Core-Elementen - nur für Titeldaten!)
 - RDFxml (RDF - Linked Data Service)

Eine Suchanfrage nach Titeln, die das Suchwort "Klimawandel" enthalten und im Jahr 2005 erschienen sind und die im Format DNB Casual ausgegeben werden soll, sieht dann folgendermaßen aus: 

In [5]:
#Parameter, die wir mit einer Anfrage übergeben wollen: 
parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : 'tit=Klimawandel and jhr=2005',
             'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

r = requests.get(dnb_url, params = parameter)

#Parsen der Antwort "r" als XML in die neue Variable "response":
response = soup(r.content)

#Schöne Ausgabe der ersten 1000 Zeichen: 
print(response.prettify()[0:1000])


<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <searchretrieveresponse xmlns="http://www.loc.gov/zing/srw/">
   <version>
    1.1
   </version>
   <numberofrecords>
    27
   </numberofrecords>
   <records>
    <record>
     <recordschema>
      oai_dc
     </recordschema>
     <recordpacking>
      xml
     </recordpacking>
     <recorddata>
      <dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
       <dc:title>
        Anpassung an den Klimawandel : Gründe, Folgen, Handlungsoptionen / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung ; Gtz, Deutsche Gesellschaft für Technische Zusammenarbeit (GTZ) GmbH
       </dc:title>
       <dc:creator>
        Deutschland / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung
       </dc:creator>
       <dc:publisher>
        Eschborn : GTZ
       </dc

In Zeile 8 der Antwort entdecken wir folgenden XML-Block, der uns die Gesamtzahl der gefundenen Ergebnisse verrät:  
```
<numberofrecords>
    27
</numberofrecords>
```

Wenn wir nicht unbedingt im XML nach diesem Schnipsel suchen wollen, können wir auch unseren Code damit beauftragen, den entsprechenden Abschnitt mithilfe des Zusatzes ".find('numberofrecords')" zu suchen. Der Zusatz ".find('suchtext')" sucht dabei nach dem ersten Element, welches den Suchtext in Klammern enthält, was für unseren Fall der Gesamtergebnisse ausreichend ist. Wenn dagegen mehrere XML-Tags mit demselben Namen gesucht und ausgegeben werden sollen, nutzt man ".find_all('suchtext')". 

Im Anschluss lassen wir uns den Inhalt zwischen den beiden "numberofrecords"-Tags ausgeben, indem wir an unsere Variable das Attribut ".text" anhängen. Zum Vergleich kann die gesamte Variable "number" (unten auskommentiert) ausgegeben werden:

In [6]:
number = response.find('numberofrecords')
print(number.text, 'Ergebnisse')
#print(number)


27 Ergebnisse


Da die einzelnen Treffer bzw. Werke jeweils durch "record"-Tags gekennzeichnet sind, suchen wir nun nach diesen und speichern sie in der Variable "records" zwischen. Zum Vergleich lassen wir uns im Anschluss noch die Länge der Variable ausgeben und sehen, dass diese mit der Angabe unter "numberofrecords" übereinstimmt. 

ACHTUNG: Dies funkioniert nur bis zu einer Treffermenge von insgesamt 100! Auch bei größeren Treffermengen wird im folgenden maximal die Länge 100 angezeigt - wie wir damit umgehen, folgt weiter unten. 

In [7]:
records = response.find_all('record')
print(len(records), 'Ergebnisse')

27 Ergebnisse


Die Ergebnisse werden als Liste gespeichert, was bedeutet, dass unsere Variable "records" eine Listenvariable ist. Die Ergebnisse stehen dabei jeweils an einem eigenen Listenplatz - bei 9 Ergebnissen gibt es in unserer Liste also 9 Einträge. Da der erste Eintrag allerdings an Listenplatz 0 steht, sind die Plätze von 0-8 mit den 9 Einträgen belegt - dies ist wichtig, wenn wir einzelene Listenplätze adressieren wollen. 

Wir können uns so bspw. den 3. Einträg anzeigen lassen, in dem wir uns den Platz mit der Nummer 2 aufrufen: 

In [61]:
print(records[5])

<record><recordschema>oai_dc</recordschema><recordpacking>xml</recordpacking><recorddata><dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
<dc:title>Informationssystem KLARA 1.0 : Klimawandel - Auswirkungen, Risiken, Anpassung ; Analysen spezifischer Verwundbarkeiten und Handlungsoptionen im Land Baden-Württemberg / LfU, Landesanstalt für Umweltschutz Baden-Württemberg ...  Projektleiter: Manfred Stock ...</dc:title>
<dc:creator>Stock, Manfred [Mitwirkender]</dc:creator>
<dc:publisher>Karlsruhe : LfU</dc:publisher>
<dc:date>2005</dc:date>
<dc:identifier xsi:type="dnb:IDN">97609228X</dc:identifier>
<dc:subject>330 Wirtschaft</dc:subject>
<dc:subject>360 Soziale Probleme, Sozialdienste, Versicherungen</dc:subject>
<dc:subject>550 Geowissenschaften</dc:subject>
<dc:format>1 CD-ROM</dc:format>
</dc></recorddata><recordposition>6</recordposit

Um das Ganze nun etwas bequemer nutzen zu können, führen wir die verschiedenen einzelnen Schritte nun in einer Funktion zusammen, die wir dann nur noch mit unserer Wunschabfrage aufrufen müssen, um unsere Ergebnisse zu erhalten: 

In [94]:
def sru_dnb(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content)
    records = response.find_all('record')  
    
    return records


In [97]:
myquery = sru_dnb('tit=Klimawandel and jhr=2005')
print(len(myquery), "Ergebnisse")

27 Ergebnisse


Da wir allerdings im Vorfeld meist nicht wissen, wieviele Ergebnisse unsere Suchanfrage erzeugen wird und wir damit rechnen, dass häufig Ergebnismengen von über 100 Treffern vorkommen, passen wir die Funktion nun noch etwas an. Dazu bauen wir eine kleine Schleife, die zunächst abfragt, wieviele Treffer gefunden wurden und dann entscheidet: Werden bis zu 100 Treffer gemeldet gibt es keinen Änderungsbedarf und die Funktion kann so einfach durchlaufen. Werden jedoch mehr als 100 Treffer gefunden, wird die Anfrage in 100er Schritte aufgeteilt und die Ergebnisse jeweils zwischengespeichert. Sobald alle Teile der Anfrage verarbeitet wurden, wird dann das gesammelte Ergebnis übergeben. Die Funktion sieht dann folgendermaßen aus: 

In [99]:
def sru_dnb(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content)
    records = response.find_all('record')  
    
    
    if len(records) < 100:
        return records
    
    else: 
        num_results = 100
        i = 101
        
        while num_results == 100:
            
            parameter.update({'startRecord': i})
            r = requests.get(dnb_url, params=parameter)
            xml = soup(r.content)
            new_records = xml.find_all('record')
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

Diese Funktion wird auch bereits im Tutorial ["Wie können Daten mittels der SRU-Schnittstelle abgerufen werden?"](https://www.dnb.de/DE/Professionell/Services/WissenschaftundForschung/DNBLab/dnblab_node.html#doc731014bodyText4) verwendet. 

Eine Abfrage für das Titelstichwort "Klimawandel", kombiniert nun mit den Jahr 2019, ergibt nun 272 Ergebnisse: 

In [101]:
myquery = sru_dnb('tit=Klimawandel and jhr=2019')
print(len(myquery), "Ergebnisse")

273 Ergebnisse


### 4. Verarbeiten der Ergebnisse  <a class="anchor" id="Teil4"></a>

Um mit diesen Ergebnissen nun weiterarbeiten zu können, überführen wir diese aus dem XML in eine tabellarische Form. 

Auch hier arbeiten wir wieder mit einer Funktion, um diese für verschiedene Anfragen nachnutzen zu können. Wir nennen diese Funktion "parse_record", weil wir ihr beim späteren Aufruf die einzelnen Records, die wir durch unsere Anfrage an die SRU-Schnittstelle übergeben werden. Die Funktion soll dann nach einem bestimmten Feld suchen (bspw. Creator, Titel etc.) und uns den Inhalt dieses Feldes zurückgeben. 

Hierfür geben wir den Namespace unseres XML an - dieser ist in der XML-Antwort der Schnittstelle zu finden. Außerdem parsen wir unsere Antwort mit Hilfe der Bibliothek ElementTree (hier kurz ET) in die passende Form zur Weiterverarbeitung. Danach definieren wir, wo der Titel zu finden ist (im Feld beginnend mit "dc:title") und holen uns von dort den Inhalt (den Titeltext). Wenn die Funktion dort nichts findet, gibt sie das des Titeltextes den Text "unknown" zurück:   

In [107]:
def parse_record(record):
    
    ns = {"dc": "http://purl.org/dc/elements/1.1/"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    #titel
    titel = xml.xpath('.//dc:title', namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
        
        
    meta_dict = {"titel":titel}
    
    return meta_dict

Im Anschluss führen wir nun die Funktion aus, in dem wir ihr die einzelnen records aus unserer Antwort "myquery" übergeben. Das Ergebnis ("output") überführen wir dann in ein Dataframe ("df") und lassen uns dieses anzeigen:

In [114]:
output = [parse_record(record) for record in myquery]
df = pd.DataFrame(output)
df

,titel
0,12. Mitteldeutscher Rinder-Workshop in Bernbur...
1,"25. Welt-Klimakonferenz in Madrid, deutschland..."
2,"55 knackige Punkte, wie wir gemeinsam den Klim..."
3,"55 knackige Punkte, wie wir gemeinsam den Klim..."
4,Adaptive potential of the Arctic diatom Thalas...
...,...
268,Der Klimawandel wird weitergehen — eine unbequ...
269,"Fortbildungsangebote zu Klimawandel, Hitze und..."
270,"Globalisierung, Klimawandel, Migration / von red"
271,Nachfrageprognose und Wasserverbrauchssteuerun...


Dieses können wir uns nun als CSV-Datei oder auch direkt im Excel-Format xlsx ausgeben lassen: 

In [115]:
df.to_csv("Titelabfrage.csv", index=False) 
#df.to_excel("Titelabfrage.xlsx", encoding='utf8')

Da wir allerdings nicht nur den Titel in unsere Tabelle bzw. Dataframe überführen wollen, sondern weitere Angaben wie IDN, Creator, Sachschlagworte, Publikationsdatum etc. erweitern wir die Funktion entsprechend. Das Prinzip bleibt dabei gleich. Da es jedoch Felder in Dublin Core gibt, die sich wiederholen können, wurde im Folgenden der Code für die Schlagworte (subject) so angepasst, dass für jeden Titel bis zu drei Schlagworte aus den Daten extrahiert werden können:  

In [121]:
def parse_record(record):
    
    ns = {"dc": "http://purl.org/dc/elements/1.1/", 
          "xsi": "http://www.w3.org/2001/XMLSchema-instance"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath(".//dc:identifier[@xsi:type='dnb:IDN']", namespaces=ns) 
    if idn: 
        idn = idn[0].text
    else:
        idn = 'fail'
    
    
    #titel
    titel = xml.xpath('.//dc:title', namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
        
    
    #creator
    creator = xml.xpath('.//dc:creator', namespaces=ns)
    try:
        creator = creator[0].text
    except:
        creator = "unkown"
        
    
    #date
    date = xml.xpath('.//dc:date', namespaces=ns)
    try:
        date = date[0].text
    except:
        date = "unkown"
        
        
        
    #subjects: 
    subject = xml.xpath(".//dc:subject", namespaces=ns) 
    
    if len(subject) == 1: 
        subject1 = subject[0].text
        subject2 = 'N/A'
        subject3 = 'N/A'
    elif len(subject) == 2:
        subject1 = subject[0].text
        subject2 = subject[1].text
        subject3 = 'N/A'
    elif len(subject) == 3:
        subject1 = subject[0].text
        subject2 = subject[1].text
        subject3 = subject[2].text
    else:
        subject1 = 'N/A'
        subject2 = 'N/A'
        subject3 = 'N/A'
        
        
    #weitere IDs: 
    urn = xml.xpath(".//dc:identifier[@xsi:type='tel:URN']", namespaces=ns) 
    if urn: 
        urn = urn[0].text
    else:
        urn = 'N/A'
        
        
    meta_dict = {"idn":idn, "titel":titel, "creator":creator, "date":date, 
                 "subject1":subject1, "subject2":subject2, "subject3":subject3, "urn":urn}
    
    return meta_dict

In [118]:
output = [parse_record(record) for record in myquery]
df = pd.DataFrame(output)
df

,idn,titel,creator,date,subject1,subject2,subject3,urn
0,1206546913,12. Mitteldeutscher Rinder-Workshop in Bernbur...,"Scholz, Heiko [Herausgeber]",2019,"630 Landwirtschaft, Veterinärmedizin",N/A,N/A,N/A
1,1206340592,"25. Welt-Klimakonferenz in Madrid, deutschland...",unkown,2019,ERROR,ERROR,ERROR,N/A
2,1176214233,"55 knackige Punkte, wie wir gemeinsam den Klim...","Matthée, Jörg",2019,"300 Sozialwissenschaften, Soziologie, Anthropo...",N/A,N/A,N/A
3,117864622X,"55 knackige Punkte, wie wir gemeinsam den Klim...","Matthée, Jörg [Verfasser]",2019,330 Wirtschaft,N/A,N/A,urn:nbn:de:101:1-2019022020104008152243
4,1186248742,Adaptive potential of the Arctic diatom Thalas...,"Wolf, Klara Katharina Estrella [Verfasser]",2019,"570 Biowissenschaften, Biologie",N/A,N/A,urn:nbn:de:gbv:46-00107155-14
...,...,...,...,...,...,...,...,...
268,1212955544,Der Klimawandel wird weitergehen — eine unbequ...,"Neubäumer, Renate [Verfasser]",2019,330 Wirtschaft,N/A,N/A,urn:nbn:de:101:1-2020070112021412765574
269,118985712X,"Fortbildungsangebote zu Klimawandel, Hitze und...","Schoierer, Julia [Verfasser]",2019,"610 Medizin, Gesundheit",N/A,N/A,urn:nbn:de:101:1-2019070412392166110088
270,1184762767,"Globalisierung, Klimawandel, Migration / von red",red [Verfasser],2019,"610 Medizin, Gesundheit",N/A,N/A,urn:nbn:de:101:1-2019042819000522717073
271,1177804212,Nachfrageprognose und Wasserverbrauchssteuerun...,"Yildiz, Özgür [Verfasser]",2019,330 Wirtschaft,N/A,N/A,urn:nbn:de:101:1-2019021204072972329687


Damit ist unsere Abfrage deutlich erweitert und wir können sie wieder als CSV exportieren oder auch direkt in Python mit dem Dataframe weiterarbeiten.

In [120]:
df.to_csv("Titelabfrage_erweitert.csv", index=False, encoding='utf8') 

In [122]:
df.to_excel("Titelabfrage_erweitert.xlsx", index=False, encoding='utf8') 